Author: Waheed Zarif

Date: 07082020

Version: 0.1

Description: 
    In this excersize, I use a dataset form Lending Club which releases data of all the approved or denied loan applications
    on thier platforms. This dataset is from 2007-2011, that's because all the loans are either paid off or defaulted

In [155]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline


In [156]:
loans = pd.read_csv('data.csv')
loans.shape

(42542, 150)

## Cleaning the data

In [157]:
drop_cols = ['desc', 'url']
cleaned_loans = loans.drop(drop_cols, axis = 1)
cleaned_loans.shape

(42542, 148)

Counting missing values in each column
If 50% or more of the rows in a column has missing values, we drop from the data frame

In [158]:
isnull_count = cleaned_loans.isnull().sum()
half_df_size = round(cleaned_loans.shape[0]*0.5)
drop_cols = isnull_count[isnull_count>=half_df_size].index
cleaned_loans = cleaned_loans.drop(drop_cols, axis=1)
cleaned_loans.shape

(42542, 57)

Displaying the remaining column names 

In [159]:
for i in cleaned_loans.columns:
    print(i)

id
loan_amnt
funded_amnt
funded_amnt_inv
term
int_rate
installment
grade
sub_grade
emp_title
emp_length
home_ownership
annual_inc
verification_status
issue_d
loan_status
pymnt_plan
purpose
title
zip_code
addr_state
dti
delinq_2yrs
earliest_cr_line
fico_range_low
fico_range_high
inq_last_6mths
open_acc
pub_rec
revol_bal
revol_util
total_acc
initial_list_status
out_prncp
out_prncp_inv
total_pymnt
total_pymnt_inv
total_rec_prncp
total_rec_int
total_rec_late_fee
recoveries
collection_recovery_fee
last_pymnt_d
last_pymnt_amnt
last_credit_pull_d
last_fico_range_high
last_fico_range_low
collections_12_mths_ex_med
policy_code
application_type
acc_now_delinq
chargeoff_within_12_mths
delinq_amnt
pub_rec_bankruptcies
tax_liens
hardship_flag
debt_settlement_flag


Columns descriptions are found here. 
https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit#gid=2081333097

Some column values are either useless for machine learning purposes or they leak data about the target the column. These are as following: 

    id: randomly generated field by Lending Club for unique identification purposes only
    member_id: also a randomly generated field by Lending Club for unique identification purposes only
    funded_amnt: leaks data from the future (after the loan is already started to be funded)
    funded_amnt_inv: also leaks data from the future (after the loan is already started to be funded)
    grade: contains redundant information as the interest rate column (int_rate)
    sub_grade: also contains redundant information as the interest rate column (int_rate)
    emp_title: requires other data and a lot of processing to potentially be useful
    issue_d: leaks data from the future (after the loan is already completely funded)
    zip_code: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
    out_prncp: leaks data from the future, (after the loan already started to be paid off)
    out_prncp_inv: also leaks data from the future, (after the loan already started to be paid off)
    total_pymnt: also leaks data from the future, (after the loan already started to be paid off)
    total_pymnt_inv: also leaks data from the future, (after the loan already started to be paid off)
    total_rec_prncp: also leaks data from the future, (after the loan already started to be paid off)
    total_rec_int: leaks data from the future, (after the loan already started to be paid off),
    total_rec_late_fee: also leaks data from the future, (after the loan already started to be paid off),
    recoveries: also leaks data from the future, (after the loan already started to be paid off),
    collection_recovery_fee: also leaks data from the future, (after the loan already started to be paid off),
    last_pymnt_d: also leaks data from the future, (after the loan already started to be paid off),
    last_pymnt_amnt: also leaks data from the future, (after the loan already started to be paid off)

The above columns are drop from the dataframe

In [160]:
cols_to_drop = [
    'id', 'funded_amnt', 'funded_amnt_inv', 'grade', 'sub_grade', 'emp_title', 'issue_d',
    'zip_code', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
    'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'last_pymnt_d',   
]
cleaned_loans = cleaned_loans.drop(cols_to_drop, axis = 1)

In [165]:
cleaned_loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42542 entries, 0 to 42541
Data columns (total 38 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   42535 non-null  float64
 1   term                        42535 non-null  object 
 2   int_rate                    42535 non-null  object 
 3   installment                 42535 non-null  float64
 4   emp_length                  41423 non-null  object 
 5   home_ownership              42535 non-null  object 
 6   annual_inc                  42531 non-null  float64
 7   verification_status         42535 non-null  object 
 8   loan_status                 42535 non-null  object 
 9   pymnt_plan                  42535 non-null  object 
 10  purpose                     42535 non-null  object 
 11  title                       42522 non-null  object 
 12  addr_state                  42535 non-null  object 
 13  dti                         425

The best candidate for the target column is the loan status column, which shows us if a  loan is paid or not

In [169]:
cleaned_loans['loan_status'].value_counts()

Fully Paid                                             34116
Charged Off                                             5670
Does not meet the credit policy. Status:Fully Paid      1988
Does not meet the credit policy. Status:Charged Off      761
Name: loan_status, dtype: int64

Fully Paid	= Loan has been fully paid off.
Charged Off	= Loan for which there is no longer a reasonable expectation of further payments.
Does not meet the credit policy. Status:Fully Paid	= While the loan was paid off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.
Does not meet the credit policy. Status:Charged Off	= While the loan was charged off, the loan application today 

So in this case, we are only going to keep the rows that shows either fully paid or charged off. Rest of the rows are dropped. 

Also, we will change 'Fully Paid' to 1, and 'Charged Off' to 0, making a numerical value that can be used by machine learning algorithm. 

In [173]:
cleaned_loans = cleaned_loans[(cleaned_loans['loan_status'] == 'Fully Paid') | 
                              (cleaned_loans['loan_status'] == 'Charged Off')]
map_dict = {
    'loan_status': {
        'Fully Paid': 1,
        'Charged Off': 0    
    }
}

cleaned_loans = cleaned_loans.replace(map_dict)
print(cleaned_loans.shape)
cleaned_loans['loan_status'].value_counts()

(39786, 38)


1    34116
0     5670
Name: loan_status, dtype: int64

Are there any columns with only one unique value (discounting NaN values as unique)? 

We will drop any columns with one unique value because it doesn't add any useful information to the model

In [195]:
cols_to_drop = []
for columns in cleaned_loans:
    non_null = cleaned_loans[columns].dropna()
    unique_non_null = len(non_null.unique())
    if unique_non_null == 1:
        cols_to_drop.append(columns)
    else:
        pass

cleaned_loans = cleaned_loans.drop(cols_to_drop, axis=1)

In [197]:
cleaned_loans.shape

(39786, 28)

## Preparing Features

In [199]:
null_count = cleaned_loans.isnull().sum()
null_count[null_count>0]

emp_length              1078
title                     11
revol_util                50
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64

Although emp_length, which is employement length, has many missing values, it is an important column to keep
The other column with a lot of missing values is pub_rec_bankruptcies, which is Public Record of Bankruptcy. 
Let's do a requency count to see the distribution in this column

In [201]:
cleaned_loans['pub_rec_bankruptcies'].value_counts(normalize=True, dropna=False)

0.0    0.940130
1.0    0.042176
NaN    0.017519
2.0    0.000176
Name: pub_rec_bankruptcies, dtype: float64

Seems like ~95% of the data is the 0 category. Other category values are marginal. 
So this column doesn't add alot of value to the algorithm. It will be droped

In [202]:
cleaned_loans = cleaned_loans.drop(['pub_rec_bankruptcies'], axis=1)

For the rest of the columns with missing values, we will keep them but drop the rows that have missing values 

In [205]:
cleaned_loans = cleaned_loans.dropna()

What is the data type distribution of the dataframe?

In [207]:
cleaned_loans.dtypes.value_counts()

float64    14
object     12
int64       1
dtype: int64

Floats and integers can be used natively by the algorithm, but object types need to be converted. 
Which are the object columns? 

In [210]:
object_columns_df = cleaned_loans.select_dtypes(include=['object'])
object_columns_df.columns

Index(['term', 'int_rate', 'emp_length', 'home_ownership',
       'verification_status', 'purpose', 'title', 'addr_state',
       'earliest_cr_line', 'revol_util', 'last_credit_pull_d',
       'debt_settlement_flag'],
      dtype='object')

Some columns represent categorical values:

        home_ownership: home ownership status, can only be 1 of 4 categorical values according to the data dictionary,
        verification_status: indicates if income was verified by Lending Club,
        emp_length: number of years the borrower was employed upon time of application,
        term: number of payments on the loan, either 36 or 60,
        addr_state: borrower's state of residence,
        purpose: a category provided by the borrower for the loan request,
        title: loan title provided by the borrower,
        
There are also some columns that represent numeric values, that need to be converted:

        int_rate: interest rate of the loan in %,
        revol_util: revolving line utilization rate or the amount of credit the borrower is using relative to all available credit
    
    
Some of the columns contain date values that would require a good amount of feature engineering for them to be potentially useful:

    earliest_cr_line: The month the borrower's earliest reported credit line was opened,
    last_credit_pull_d: The most recent month Lending Club pulled credit for this loan.
Since these date features require some feature engineering for modeling purposes, they will be removed

In [214]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state','title', 'purpose']
for i in cols:
    print(loans[i].value_counts(),'\n\n')

RENT        20181
MORTGAGE    18959
OWN          3251
OTHER         136
NONE            8
Name: home_ownership, dtype: int64 


Not Verified       18758
Verified           13471
Source Verified    10306
Name: verification_status, dtype: int64 


10+ years    9369
< 1 year     5062
2 years      4743
3 years      4364
4 years      3649
1 year       3595
5 years      3458
6 years      2375
7 years      1875
8 years      1592
9 years      1341
Name: emp_length, dtype: int64 


 36 months    31534
 60 months    11001
Name: term, dtype: int64 


CA    7429
NY    4065
FL    3104
TX    2915
NJ    1988
IL    1672
PA    1651
GA    1503
VA    1487
MA    1438
OH    1329
MD    1125
AZ     933
WA     888
CO     857
NC     830
CT     816
MI     796
MO     765
MN     652
NV     527
WI     516
SC     489
AL     484
OR     468
LA     461
KY     359
OK     317
KS     298
UT     278
AR     261
DC     224
RI     208
NM     205
NH     188
WV     187
HI     181
DE     136
MT      96
WY      87
AK      86
SD 


The home_ownership, verification_status, emp_length, and term columns each contain a few discrete categorical values. We should encode these columns as dummy variables and keep them.

It seems like the purpose and title columns do contain overlapping information but we'll keep the purpose column since it contains a few discrete values. In addition, the title column has data quality issues since many of the values are repeated with slight modifications (e.g. Debt Consolidation and Debt Consolidation Loan and debt consolidation).

The addr_state column contains many discrete values and we'd need to add 49 dummy variable columns to use it for classification. This would make our Dataframe much larger and could slow down how quickly the code runs. Let's remove this column from consideration.

So we will drop the following columns:


In [215]:
col_drop = ['last_credit_pull_d', 'addr_state', 'title', 'earliest_cr_line']
cleaned_loans = cleaned_loans.drop(col_drop, axis=1)

We will also use map to clean the emp_lenght column

In [216]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}
cleaned_loans = cleaned_loans.replace(mapping_dict)

Lastly, will clean the int_rate and revol_util rate by removing the percentage sign and change the value to integer

In [217]:
col_to_int = ['int_rate', 'revol_util']
for i in col_to_int:
    cleaned_loans[i] = cleaned_loans[i].str.rstrip('%')
    cleaned_loans[i] = pd.to_numeric(cleaned_loans[i])

Let's now encode the home_ownership, verification_status, purpose, and term columns as dummy variables so we can use them in our model. 

In [219]:
cols_dummy = ['home_ownership', 'verification_status', 'purpose', 'term']
dummy_df = pd.get_dummies(cleaned_loans[cols_dummy])
cleaned_loans = pd.concat([cleaned_loans, dummy_df], axis =1 )
cleaned_loans = cleaned_loans.drop(cols_dummy, axis=1)